<a href="https://colab.research.google.com/github/sumedhgodbole/guessthepin/blob/main/guess_the_pin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import itertools
from tqdm import tqdm
import multiprocessing as mp

In [ ]:
!apt-get update
!pip install selenium
!apt install chromium-chromedriver
# !apt install firefox-geckodriver
# !cp /usr/lib/geckodriver /usr/bin
# !cp /usr/lib/firefox /usr/bin
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options


In [ ]:
url = "https://www.guessthepin.com/"

In [ ]:
number_of_parts = 10
total_combinations = 10000
chunk_size = total_combinations //number_of_parts
iterator = [i for i in range(0, 10)]

In [ ]:
digits = [str(d) for d in range(10)]
# print(len(digits), digits)
keys = ["".join(item) for item in itertools.product(digits, repeat=4)]

In [ ]:
from selenium.webdriver.firefox.options import Options
def guess_the_pin(seq_number):

    range_start, range_end = (chunk_size * seq_number ) +1, (chunk_size * (seq_number+1)) +1

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    driver.get(url)

    inputElement = driver.find_element(By.ID, "pin")
    inputElement.send_keys('0000')

    inputElement.send_keys(Keys.ENTER)

    # # time.sleep(1.0)
    label_ret = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//div[@id='container']/form/label"))).text.split()
    for k in tqdm(keys[range_start:range_end], leave=True, position=0):

        if label_ret[0] == "Sorry,":

            inputElement = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "pin")))
            inputElement.send_keys(k)
            inputElement.send_keys(Keys.ENTER)
            expected_update = "Sorry, {}".format(k)
            # print(expected_update)
            text_present = WebDriverWait(driver, 20).until(
                EC.text_to_be_present_in_element((By.XPATH, "//div[@id='container']/form/label"), expected_update))

        else:
            print("Key Found is {}", label_ret.split()[0:2])
            break

In [ ]:
if __name__ == '__main__':
    pool = mp.Pool(processes=mp.cpu_count())
    pool.map(guess_the_pin, iterator)

In [ ]:
# guess_the_pin(2)